# 데이터 로드 및 사전처리
## 1. CSV 데이터 로드
#### 참고사이트 : https://www.tensorflow.org/tutorials/load_data/csv?hl=ko

In [8]:
import functools

import numpy as np
import tensorflow as tf

In [9]:
TRAIN_DATA_URL = "https://storage.googleapis.com/tf-datasets/titanic/train.csv"
TEST_DATA_URL = "https://storage.googleapis.com/tf-datasets/titanic/eval.csv"

train_file_path = tf.keras.utils.get_file("train.csv", TRAIN_DATA_URL)
test_file_path = tf.keras.utils.get_file("eval.csv", TEST_DATA_URL)

In [10]:
# Make numpy values easier to read.
np.set_printoptions(precision=3, suppress=True)

In [11]:
# get_file을 통해 저장되는 위치는 ~/.keras/datasets/
# csv 파일의 상단(head) 을 보고 형식이 어떻게 지정되었는지 확인
!head {train_file_path} 

survived,sex,age,n_siblings_spouses,parch,fare,class,deck,embark_town,alone
0,male,22.0,1,0,7.25,Third,unknown,Southampton,n
1,female,38.0,1,0,71.2833,First,C,Cherbourg,n
1,female,26.0,0,0,7.925,Third,unknown,Southampton,y
1,female,35.0,1,0,53.1,First,C,Southampton,n
0,male,28.0,0,0,8.4583,Third,unknown,Queenstown,y
0,male,2.0,3,1,21.075,Third,unknown,Southampton,n
1,female,27.0,0,2,11.1333,Third,unknown,Southampton,n
1,female,14.0,1,0,30.0708,Second,unknown,Cherbourg,n
1,female,4.0,1,1,16.7,Third,G,Southampton,n


In [12]:
# 결국 식별해야 할 것은 생존 가능성인 survived 열
LABEL_COLUMN = 'survived'
LABELS = [0, 1]

In [13]:
# file에서 csv 데이터를 읽고 dataset으로 만들기
# tf.data.experimental.make_csv_dataset() 함수 확인
# https://www.tensorflow.org/api_docs/python/tf/data/experimental/make_csv_dataset?hl=ko
def get_dataset(file_path, **kwargs):
  dataset = tf.data.experimental.make_csv_dataset(
      file_path,
      batch_size=5, # 예제를 쉽게 보여주기 위해 일부러 사이즈 작게 설정
      label_name=LABEL_COLUMN,
      na_value="?",
      num_epochs=1,
      ignore_errors=True, 
      **kwargs)
  return dataset

raw_train_data = get_dataset(train_file_path)
raw_test_data = get_dataset(test_file_path)

Instructions for updating:
Use `tf.data.Dataset.interleave(map_func, cycle_length, block_length, num_parallel_calls=tf.data.experimental.AUTOTUNE)` instead. If sloppy execution is desired, use `tf.data.Options.experimental_determinstic`.


In [14]:
def show_batch(dataset):
  for batch, label in dataset.take(1):
    for key, value in batch.items():
      print("{:20s}: {}".format(key,value.numpy()))

In [15]:
# dataset의 항목은 batch 값대로 5개가 tuple로 표시
# 이 예제는 열(row) 기반 tensor이며 각 행은 batch 크기만큼 요소를 가짐.
show_batch(raw_train_data)

sex                 : [b'male' b'male' b'female' b'female' b'male']
age                 : [35. 26. 45. 27.  4.]
n_siblings_spouses  : [0 0 1 0 4]
parch               : [0 0 1 0 1]
fare                : [ 26.288  10.5   164.867  10.5    29.125]
class               : [b'First' b'Second' b'First' b'Second' b'Third']
deck                : [b'E' b'unknown' b'unknown' b'E' b'unknown']
embark_town         : [b'Southampton' b'Southampton' b'Southampton' b'Southampton' b'Queenstown']
alone               : [b'y' b'y' b'n' b'y' b'n']


In [16]:
# 위의 경우처럼 자동으로 csv의 열 이름이 지정되지만, 
# 만약 첫번째 행에 열 이름이 포함되어있지 않으면 인수로 전달 가능
CSV_COLUMNS = ['survived', 'sex', 'age', 'n_siblings_spouses', 'parch', 'fare', 'class', 'deck', 'embark_town', 'alone']

temp_dataset = get_dataset(train_file_path, column_names=CSV_COLUMNS)

show_batch(temp_dataset)

sex                 : [b'male' b'male' b'male' b'male' b'female']
age                 : [28. 22. 26. 45. 19.]
n_siblings_spouses  : [1 0 2 0 1]
parch               : [2 0 0 0 0]
fare                : [23.45   9.     8.663  8.05  26.   ]
class               : [b'Third' b'Third' b'Third' b'Third' b'Second']
deck                : [b'unknown' b'unknown' b'unknown' b'unknown' b'unknown']
embark_town         : [b'Southampton' b'Southampton' b'Southampton' b'Southampton'
 b'Southampton']
alone               : [b'n' b'y' b'n' b'y' b'n']


In [17]:
# 특정 열만 출력하고 싶으면 select_columns 옵션을 통해 인수로 전달 가능
SELECT_COLUMNS = ['survived', 'age', 'n_siblings_spouses', 'class', 'deck', 'alone']

temp_dataset = get_dataset(train_file_path, select_columns=SELECT_COLUMNS)

show_batch(temp_dataset)

age                 : [49. 23. 35. 28. 30.]
n_siblings_spouses  : [1 0 0 0 3]
class               : [b'First' b'Second' b'Third' b'First' b'Second']
deck                : [b'C' b'D' b'unknown' b'A' b'unknown']
alone               : [b'n' b'y' b'y' b'y' b'n']


### 데이터 전처리

CSV 파일은 다양한 데이터 유형을 포함 할 수 있음.   
아래 예제는 데이터를 모델에 공급하기 전에 고정길이 벡터로 변환하고자 함.   

Tensorflow에는 일반적인 입력 변환을 위한 [feature_column()](https://www.tensorflow.org/tutorials/structured_data/feature_columns?hl=ko) 함수를 제공.

원하는 도구 ([nltk](https://www.nltk.org/) 또는 [sklearn](https://scikit-learn.org/stable/))를 사용하여 데이터를 사전 처리 하고 처리 된 출력을 TensorFlow에 전달.

모델 내에서 전처리를 수행 할 때의 이점은 모델을 내보낼 때 전처리를 포함한다는 것.   
이렇게 하면 원시 데이터를 모델로 직접 전달할 수 있음.

In [18]:
# 연속 데이터
# 데이터가 이미 비슷한 숫자 형식 일 경우 모델로 data 전달 전 vector로 묶을 수 있음
SELECT_COLUMNS = ['survived', 'age', 'n_siblings_spouses', 'parch', 'fare']
DEFAULTS = [0, 0.0, 0.0, 0.0, 0.0]
temp_dataset = get_dataset(train_file_path, 
                           select_columns=SELECT_COLUMNS,
                           column_defaults = DEFAULTS)

show_batch(temp_dataset)

age                 : [32. 50. 28. 36. 38.]
n_siblings_spouses  : [0. 0. 1. 1. 1.]
parch               : [0. 0. 0. 1. 0.]
fare                : [ 7.75  10.5   15.5   24.15  71.283]


In [19]:
# 모든 열을 묶는 간단한 함수
example_batch, labels_batch = next(iter(temp_dataset)) 

In [25]:
print(example_batch)
print(labels_batch)

OrderedDict([('age', <tf.Tensor: id=429, shape=(5,), dtype=float32, numpy=array([34.  , 28.  ,  0.83, 28.  , 14.  ], dtype=float32)>), ('n_siblings_spouses', <tf.Tensor: id=431, shape=(5,), dtype=float32, numpy=array([1., 0., 0., 0., 1.], dtype=float32)>), ('parch', <tf.Tensor: id=432, shape=(5,), dtype=float32, numpy=array([0., 0., 2., 0., 0.], dtype=float32)>), ('fare', <tf.Tensor: id=430, shape=(5,), dtype=float32, numpy=array([26.   ,  6.95 , 29.   , 35.5  , 11.242], dtype=float32)>)])
tf.Tensor([0 0 1 1 1], shape=(5,), dtype=int32)


In [26]:
def pack(features, label):
  return tf.stack(list(features.values()), axis=-1), label

In [27]:
# data set의 각 요소에 적용
packed_dataset = temp_dataset.map(pack)

for features, labels in packed_dataset.take(1):
  print(features.numpy()) 
  # 1번 데이터를 보면 33세 승객은 형제, 자매, 배우자가 없고 2명과 함께 탐승했고 $26을 지불함
  print()
  print(labels.numpy())
    
# 숫자, 문자 등이 섞인 혼합 데이터 유형이 있는 경우 지금처럼 단순 숫자 필드를 분리 할 수 있음.
# 하지만, 오버헤드 발생 가능성이 있기 때문에 가능한 피할 것을 추천.

[[33.     0.     2.    26.   ]
 [28.     1.     0.    51.862]
 [27.     1.     0.    13.858]
 [47.     0.     0.     9.   ]
 [22.     1.     0.     7.25 ]]

[1 1 1 0 0]


In [28]:
show_batch(raw_train_data)

sex                 : [b'male' b'female' b'male' b'male' b'female']
age                 : [28. 28. 28. 47. 36.]
n_siblings_spouses  : [8 0 0 0 0]
parch               : [2 0 0 0 0]
fare                : [ 69.55    7.787   8.05    7.25  135.633]
class               : [b'Third' b'Third' b'Third' b'Third' b'First']
deck                : [b'unknown' b'unknown' b'unknown' b'unknown' b'C']
embark_town         : [b'Southampton' b'Queenstown' b'Southampton' b'Southampton' b'Cherbourg']
alone               : [b'n' b'y' b'y' b'y' b'y']


In [29]:
example_batch, labels_batch = next(iter(temp_dataset)) 

In [30]:
class PackNumericFeatures(object):
  def __init__(self, names):
    self.names = names

  def __call__(self, features, labels):
    numeric_features = [features.pop(name) for name in self.names]
    numeric_features = [tf.cast(feat, tf.float32) for feat in numeric_features]
    numeric_features = tf.stack(numeric_features, axis=-1)
    features['numeric'] = numeric_features

    return features, labels

In [31]:
NUMERIC_FEATURES = ['age','n_siblings_spouses','parch', 'fare']

packed_train_data = raw_train_data.map(
    PackNumericFeatures(NUMERIC_FEATURES))

packed_test_data = raw_test_data.map(
    PackNumericFeatures(NUMERIC_FEATURES))

In [32]:
show_batch(packed_train_data)

sex                 : [b'male' b'female' b'male' b'male' b'male']
class               : [b'First' b'Second' b'Third' b'Second' b'First']
deck                : [b'B' b'unknown' b'unknown' b'unknown' b'B']
embark_town         : [b'Cherbourg' b'Southampton' b'Cherbourg' b'Southampton' b'Cherbourg']
alone               : [b'n' b'y' b'n' b'y' b'y']
numeric             : [[25.     1.     0.    91.079]
 [34.     0.     0.    13.   ]
 [28.     2.     0.    21.679]
 [18.     0.     0.    11.5  ]
 [32.     0.     0.    30.5  ]]


In [33]:
example_batch, labels_batch = next(iter(packed_train_data)) 

In [34]:
# 데이터 정규화
import pandas as pd
desc = pd.read_csv(train_file_path)[NUMERIC_FEATURES].describe()
desc

,age,n_siblings_spouses,parch,fare
count,627.000000,627.000000,627.000000,627.000000
mean,29.631308,0.545455,0.379585,34.385399
std,12.511818,1.151090,0.792999,54.597730
min,0.750000,0.000000,0.000000,0.000000
25%,23.000000,0.000000,0.000000,7.895800
50%,28.000000,0.000000,0.000000,15.045800
75%,35.000000,1.000000,0.000000,31.387500
max,80.000000,8.000000,5.000000,512.329200


In [35]:
MEAN = np.array(desc.T['mean'])
STD = np.array(desc.T['std'])

In [36]:
def normalize_numeric_data(data, mean, std):
  # Center the data
  return (data-mean)/std

In [37]:
# See what you just created.
normalizer = functools.partial(normalize_numeric_data, mean=MEAN, std=STD)

numeric_column = tf.feature_column.numeric_column('numeric', normalizer_fn=normalizer, shape=[len(NUMERIC_FEATURES)])
numeric_columns = [numeric_column]
numeric_column

NumericColumn(key='numeric', shape=(4,), default_value=None, dtype=tf.float32, normalizer_fn=functools.partial(<function normalize_numeric_data at 0x7fe8bb391d90>, mean=array([29.631,  0.545,  0.38 , 34.385]), std=array([12.512,  1.151,  0.793, 54.598])))

In [38]:
example_batch['numeric']

<tf.Tensor: id=550, shape=(5, 4), dtype=float32, numpy=
array([[11.   ,  4.   ,  2.   , 31.275],
       [32.   ,  0.   ,  0.   ,  8.363],
       [21.   ,  0.   ,  0.   ,  7.733],
       [32.   ,  0.   ,  0.   ,  8.05 ],
       [32.   ,  0.   ,  0.   ,  7.925]], dtype=float32)>

In [40]:
numeric_layer = tf.keras.layers.DenseFeatures(numeric_columns)
numeric_layer(example_batch).numpy()
# 평균 기반 정규화를 위해 각 열의 평균을 미리 알아야 함.

array([[-1.489,  3.001,  2.043, -0.057],
       [ 0.189, -0.474, -0.479, -0.477],
       [-0.69 , -0.474, -0.479, -0.488],
       [ 0.189, -0.474, -0.479, -0.482],
       [ 0.189, -0.474, -0.479, -0.485]], dtype=float32)

In [41]:
# 범주형 데이터
CATEGORIES = {
    'sex': ['male', 'female'],
    'class' : ['First', 'Second', 'Third'],
    'deck' : ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J'],
    'embark_town' : ['Cherbourg', 'Southhampton', 'Queenstown'],
    'alone' : ['y', 'n']
}

In [42]:
categorical_columns = []
for feature, vocab in CATEGORIES.items():
  cat_col = tf.feature_column.categorical_column_with_vocabulary_list(
        key=feature, vocabulary_list=vocab)
  categorical_columns.append(tf.feature_column.indicator_column(cat_col))

In [43]:
categorical_columns

[IndicatorColumn(categorical_column=VocabularyListCategoricalColumn(key='sex', vocabulary_list=('male', 'female'), dtype=tf.string, default_value=-1, num_oov_buckets=0)),
 IndicatorColumn(categorical_column=VocabularyListCategoricalColumn(key='class', vocabulary_list=('First', 'Second', 'Third'), dtype=tf.string, default_value=-1, num_oov_buckets=0)),
 IndicatorColumn(categorical_column=VocabularyListCategoricalColumn(key='deck', vocabulary_list=('A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J'), dtype=tf.string, default_value=-1, num_oov_buckets=0)),
 IndicatorColumn(categorical_column=VocabularyListCategoricalColumn(key='embark_town', vocabulary_list=('Cherbourg', 'Southhampton', 'Queenstown'), dtype=tf.string, default_value=-1, num_oov_buckets=0)),
 IndicatorColumn(categorical_column=VocabularyListCategoricalColumn(key='alone', vocabulary_list=('y', 'n'), dtype=tf.string, default_value=-1, num_oov_buckets=0))]

In [44]:
categorical_layer = tf.keras.layers.DenseFeatures(categorical_columns)
print(categorical_layer(example_batch).numpy()[0]) # 모델 build시 데이터 처리 입력의 일부가 됨.

Instructions for updating:
The old _FeatureColumn APIs are being deprecated. Please use the new FeatureColumn APIs instead.
Instructions for updating:
The old _FeatureColumn APIs are being deprecated. Please use the new FeatureColumn APIs instead.
[0. 1. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]


In [45]:
# 결합된 전처리 층
preprocessing_layer = tf.keras.layers.DenseFeatures(categorical_columns+numeric_columns)
print(preprocessing_layer(example_batch).numpy()[0])

[ 0.     1.     0.     0.     1.     0.     0.     0.     0.     0.
  0.     0.     0.     0.     0.     0.     0.     0.    -1.489  3.001
  2.043 -0.057  0.     1.   ]


In [46]:
# 모델 구축
model = tf.keras.Sequential([
  preprocessing_layer,
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dense(1),
])

model.compile(
    loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
    optimizer='adam',
    metrics=['accuracy'])

In [47]:
train_data = packed_train_data.shuffle(500)
test_data = packed_test_data

In [51]:
model.fit(train_data, epochs=20) # 모델 훈련(학습)시키기

Epoch 1/20
126/126 [==============================] - 1s 10ms/step - loss: 0.5196 - accuracy: 0.7209
Epoch 2/20
126/126 [==============================] - 0s 2ms/step - loss: 0.4251 - accuracy: 0.8166
Epoch 3/20
126/126 [==============================] - 0s 2ms/step - loss: 0.4065 - accuracy: 0.8214
Epoch 4/20
126/126 [==============================] - 0s 2ms/step - loss: 0.3930 - accuracy: 0.8293
Epoch 5/20
126/126 [==============================] - 0s 2ms/step - loss: 0.3876 - accuracy: 0.8246
Epoch 6/20
126/126 [==============================] - 0s 2ms/step - loss: 0.3750 - accuracy: 0.8357
Epoch 7/20
126/126 [==============================] - 0s 2ms/step - loss: 0.3671 - accuracy: 0.8373
Epoch 8/20
126/126 [==============================] - 0s 2ms/step - loss: 0.3622 - accuracy: 0.8437
Epoch 9/20
126/126 [==============================] - 0s 2ms/step - loss: 0.3628 - accuracy: 0.8421
Epoch 10/20
126/126 [==============================] - 0s 2ms/step - loss: 0.3535 - accuracy: 0.856

In [52]:
# test_data set을 통해 훈련(학습)된 데이터의 정확도 확인
test_loss, test_accuracy = model.evaluate(test_data)

print('\n\nTest Loss {}, Test Accuracy {}'.format(test_loss, test_accuracy))

53/53 [==============================] - 0s 9ms/step - loss: 0.4959 - accuracy: 0.8447


Test Loss 0.495882159977589, Test Accuracy 0.8446969985961914


In [53]:
# predict 함수를 통해 배치 데이터셋에서 레이블 유추
predictions = model.predict(test_data)

# Show some results
for prediction, survived in zip(predictions[:10], list(test_data)[0][1][:10]):
  prediction = tf.sigmoid(prediction).numpy()
  print("Predicted survival: {:.2%}".format(prediction[0]),
        " | Actual outcome: ",
        ("SURVIVED" if bool(survived) else "DIED"))


Predicted survival: 12.70%  | Actual outcome:  DIED
Predicted survival: 93.42%  | Actual outcome:  DIED
Predicted survival: 6.50%  | Actual outcome:  DIED
Predicted survival: 9.73%  | Actual outcome:  DIED
Predicted survival: 56.17%  | Actual outcome:  SURVIVED


In [54]:
print(predictions)

[[-1.928]
 [ 2.654]
 [-2.667]
 [-2.227]
 [ 0.248]
 [ 9.364]
 [-2.27 ]
 [ 1.855]
 [-2.254]
 [-1.471]
 [-1.197]
 [-2.151]
 [-1.978]
 [-0.782]
 [ 5.781]
 [-0.145]
 [ 0.4  ]
 [ 1.739]
 [-2.231]
 [-2.236]
 [ 0.153]
 [ 4.214]
 [ 6.011]
 [-1.053]
 [-1.939]
 [-2.637]
 [ 8.657]
 [ 1.384]
 [ 4.216]
 [-2.191]
 [-2.048]
 [ 2.304]
 [-0.98 ]
 [ 1.125]
 [-0.484]
 [ 3.17 ]
 [-2.266]
 [-2.908]
 [-1.464]
 [ 6.965]
 [ 9.865]
 [-0.753]
 [-2.316]
 [-1.071]
 [ 5.277]
 [-2.087]
 [ 2.851]
 [ 2.875]
 [ 5.782]
 [ 7.738]
 [-2.854]
 [ 5.536]
 [-1.844]
 [ 5.635]
 [ 6.74 ]
 [ 1.867]
 [ 1.854]
 [10.604]
 [-3.894]
 [-2.315]
 [ 2.367]
 [ 0.106]
 [-0.419]
 [ 1.59 ]
 [-2.356]
 [ 1.24 ]
 [ 0.341]
 [ 0.082]
 [ 3.52 ]
 [-3.054]
 [-1.708]
 [-4.089]
 [-0.735]
 [ 9.421]
 [-4.176]
 [ 4.492]
 [-2.089]
 [ 1.674]
 [-2.288]
 [-1.219]
 [-2.235]
 [ 4.438]
 [-4.169]
 [-0.157]
 [-1.078]
 [-2.29 ]
 [ 1.851]
 [-3.054]
 [-1.227]
 [-2.796]
 [-2.389]
 [-2.349]
 [ 0.419]
 [-1.266]
 [-2.718]
 [-2.311]
 [-2.166]
 [-2.761]
 [-2.217]
 [-2.351]
